In [1]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpaf7xx6sl".


In [6]:
%%cuda
#include <assert.h>
#include <stdio.h>
#include<cuda.h>
#include <time.h>

void prodottoCPU(float *a, float *b, float *c, int n, float *sum);

__global__ void prodottoGPU(float *a, float *b, float *c, int n);

int main(void) {
    float *a_h, *b_h, *c_d_on_h, *c_h; // host data
    float *a_d, *b_d, *c_d; // device data
    float sum_d = 0, sum_h = 0;
    int N, nBytes;
    float elapsed;
    dim3 gridDim, blockDim;

    N = 640000;
    blockDim.x = 64;

    gridDim.x = N / blockDim.x + ((N % blockDim.x) == 0 ? 0 : 1);
    printf("\n");

    nBytes = N * sizeof(float);
    a_h = (float*) malloc(nBytes);
    b_h = (float*) malloc(nBytes);
    c_d_on_h = (float*) malloc(nBytes);
    c_h = (float*) malloc(nBytes);
    cudaMalloc((void**)&a_d, nBytes);
    cudaMalloc((void**)&b_d, nBytes);
    cudaMalloc((void**)&c_d, nBytes);

    // Inizializzazione i dati
    // Inizializza la generazione random dei vettori utilizzando l'ora attuale del sistema
    srand((unsigned int) time(0));

    for (int i = 0; i < N; i++) {
        a_h[i] = rand() % 5 - 2;
        b_h[i] = rand() % 5 - 2;;
    }

    cudaMemcpy(a_d, a_h, nBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(b_d, b_h, nBytes, cudaMemcpyHostToDevice);

    // Azzeriamo il contenuto del vettore c
    memset(c_d_on_h, 0, nBytes);
    cudaMemset(c_d, 0, nBytes);

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    //invocazione del kernel
    prodottoGPU<<<gridDim, blockDim>>>(a_d, b_d, c_d, N);
    cudaMemcpy(c_d_on_h, c_d, nBytes, cudaMemcpyDeviceToHost);
    // Somma i valori del vettore c_d_on_h
    for(int i = 0; i < N; i++){
        sum_d += c_d_on_h[i];
    }

    cudaEventRecord(stop);
    cudaEventSynchronize(stop); // assicura che tutti siano arrivati all'evento stop prima di registrare il tempo
    cudaEventElapsedTime(&elapsed, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    printf("time_GPU = %f\n", elapsed);

    cudaMemcpy(c_d_on_h, c_d, nBytes, cudaMemcpyDeviceToHost);

    // Calcolo su CPU
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);
    // Calcolo prodotto
    prodottoCPU(a_h, b_h, c_h, N, &sum_h);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop); // assicura che tutti siano arrivati all'evento stop prima di registrare il tempo
    cudaEventElapsedTime(&elapsed, start, stop);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    printf("time_CPU = %f\n", elapsed);

    for (int i = 0; i < N; i++)
        assert(c_d_on_h[i] == c_h[i]);
    assert(sum_d == sum_h);

    printf("sum_h = %f\n", sum_h);
    printf("sum_d = %f\n", sum_d);

    if (N < 20) {
        for (int i = 0; i < N; i++)
            printf("a_h[%d]=%6.2f ", i, a_h[i]);
        printf("\n");
        for (int i = 0; i < N; i++)
            printf("b_h[%d]=%6.2f ", i, b_h[i]);
        printf("\n");
        for (int i = 0; i < N; i++)
            printf("c_d_on_h[%d]=%6.2f ", i, c_d_on_h[i]);
        printf("\n");
    }
    free(a_h);
    free(b_h);
    free(c_d_on_h);
    free(c_h);
    cudaFree(a_d);
    cudaFree(b_d);
    cudaFree(c_d);
    return 0;
}

// Seriale
void prodottoCPU(float *a, float *b, float *c, int n, float *sum) {
    for (int i = 0; i < n; i++) {
        c[i] = a[i] * b[i];
        *sum += c[i];
    }
}

// Parallelo
__global__ void prodottoGPU(float *a, float *b, float *c, int n) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < n) {
        c[index] = a[index] * b[index];
    }
}



time_GPU = 2.703872
time_CPU = 3.549280
sum_h = 213.000000
sum_d = 213.000000

